# Detección de Cambios con Sentinel-1 en Municipios de Casanare y Meta

## Parte 3: Análisis de Detección de Cambios

### Introducción
Este notebook implementa algoritmos de detección de cambios en series temporales de imágenes Sentinel-1 SAR. La metodología se basa en el análisis estadístico multivariado propuesto por Conradsen et al. [1] y adaptado por Canty et al. [2] para Google Earth Engine.

### Marco teórico
**Test de razón de verosimilitud (Likelihood Ratio Test)**
Para dos imágenes SAR dual-pol adquiridas en tiempos $t_1$ y $t_2$, el test de Wishart complejo usa la estadística [1]:

$$\Lambda = -L \ln \left( \frac{\det(\mathbf{\Sigma}_1 + \mathbf{\Sigma}_2)}{2^p \det(\mathbf{\Sigma}_1)^{1/2} \det(\mathbf{\Sigma}_2)^{1/2}} \right)$$

Donde $\mathbf{\Sigma}_1$ y $\mathbf{\Sigma}_2$ son las matrices de covarianza de cada fecha, $L$ es el número de looks equivalentes y $p$ el número de polarizaciones. Valores altos de $\Lambda$ señalan cambios significativos.

**Extensión a series temporales**
Para una secuencia de $N$ imágenes, el test se aplica de forma secuencial para identificar el momento del cambio [3]. Se declara un cambio en el instante $k$ si:

$$\Lambda_k > -\ln(\beta) + (N - k)\ln(\alpha)$$

con $\alpha$ el nivel de significancia, $\beta$ un factor de corrección y grados de libertad relacionados con $p$.

**Métodos complementarios**
Para ampliar la interpretación se incluyen:

- Diferencias temporales: $\Delta_{\text{VV}} = \text{VV}_{t_2} - \text{VV}_{t_1}$ y equivalentes para VH.
- Anomalías (z-scores): $$z_{t} = \frac{x_t - \mu}{\sigma}$$ comparando cada fecha con la media histórica $\mu$ y la desviación estándar $\sigma$.
- Índice NDCV: $$\text{NDCV} = \frac{\left\|\mathbf{C}_{t_2} - \mathbf{C}_{t_1}\right\|}{\left\|\mathbf{C}_{t_2} + \mathbf{C}_{t_1}\right\|}$$ con $\mathbf{C}$ el vector de backscatter (VV, VH). Valores próximos a 1 reflejan cambios de gran magnitud.

---

## 1. Carga de librerías y datos preprocesados

Importamos `ee`, `geemap`, `pandas` y los archivos generados en los notebooks anteriores. Esta práctica garantiza que el análisis se base en los mismos parámetros temporales y en la zona de estudio que ya validamos.

In [ ]:
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Inicializar Earth Engine
try:
    ee.Initialize()
    print("Earth Engine inicializado correctamente")
except:
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine autenticado e inicializado")

In [ ]:
# Cargar datos de notebooks anteriores
municipios_seleccionados = gpd.read_file("data/municipios_seleccionados.gpkg", layer="municipios")

with open('data/parametros.json', 'r') as f:
    parametros = json.load(f)

with open('data/procesamiento_info.json', 'r') as f:
    proc_info = json.load(f)

print(f"Datos cargados:")
print(f"  Municipios: {len(municipios_seleccionados)}")
print(f"  Período: {parametros['fecha_inicio']} a {parametros['fecha_fin']}")
print(f"  Composiciones disponibles: {proc_info['n_composites']}")
print(f"  Órbita: {proc_info['orbit_type']}")

In [ ]:
# Recrear AOI y colecciones de Earth Engine
def gdf_to_ee_geometry(gdf):
    geom_union = gdf.geometry.unary_union
    if geom_union.geom_type == 'Polygon':
        coords = [list(geom_union.exterior.coords)]
        return ee.Geometry.Polygon(coords)
    elif geom_union.geom_type == 'MultiPolygon':
        coords = [list(poly.exterior.coords) for poly in geom_union.geoms]
        return ee.Geometry.MultiPolygon(coords)

aoi = gdf_to_ee_geometry(municipios_seleccionados)
print("AOI recreada")

## 2. Recreación de la colección Sentinel-1 procesada

Reconstruye las funciones de preprocesamiento (conversión a dB, filtros) para que este notebook pueda ejecutarse de forma independiente. El tutorial de Earth Engine sobre detección de cambios explica cómo encapsular este flujo en funciones reutilizables [6].

In [ ]:
# Recrear el pipeline de procesamiento del Notebook 2
def process_sentinel1_collection(aoi, start_date, end_date, orbit_type):
    """Recrea la colección procesada de Sentinel-1"""
    
    # Cargar colección
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit_type))
    
    # Conversión a dB
    def to_dB(image):
        vv = image.select('VV')
        vh = image.select('VH')
        vv_db = ee.Image(10).multiply(vv.log10()).rename('VV')
        vh_db = ee.Image(10).multiply(vh.log10()).rename('VH')
        ratio = vv.divide(vh).log10().multiply(10).rename('VV_VH_ratio')
        return image.addBands(vv_db).addBands(vh_db).addBands(ratio) \
            .copyProperties(image, ['system:time_start'])
    
    # Filtro de speckle
    def apply_speckle_filter(image):
        kernel = ee.Kernel.square(radius=3, units='pixels')
        vv_f = image.select('VV').focal_median(kernel=kernel).rename('VV_filtered')
        vh_f = image.select('VH').focal_median(kernel=kernel).rename('VH_filtered')
        ratio_f = image.select('VV_VH_ratio').focal_median(kernel=kernel).rename('ratio_filtered')
        return image.addBands(vv_f).addBands(vh_f).addBands(ratio_f)
    
    s1_processed = s1.map(to_dB).map(apply_speckle_filter)
    
    return s1_processed

# Procesar colección
s1_collection = process_sentinel1_collection(
    aoi,
    parametros['fecha_inicio'],
    parametros['fecha_fin'],
    proc_info['orbit_type']
)

print(f"Colección recreada: {s1_collection.size().getInfo()} imágenes")

## 3. Método 1: análisis de diferencias temporales

Compara una imagen de referencia con una imagen objetivo mediante restas de VV, VH y del índice VV/VH. Valores altos positivos muestran incrementos de retrodispersión, mientras que valores negativos señalan reducciones, como ocurre tras una cosecha [4].

In [ ]:
# Definir períodos de referencia y análisis
# Ejemplo: comparar mismo período en años diferentes para detectar cambios agrícolas

# Período de referencia (baseline): primer semestre 2023
reference_start = '2023-01-01'
reference_end = '2023-06-30'

# Período de análisis (target): primer semestre 2024
target_start = '2024-01-01'
target_end = '2024-06-30'

print(f"Período de referencia: {reference_start} a {reference_end}")
print(f"Período de análisis: {target_start} a {target_end}")

In [ ]:
# Crear composiciones para cada período
reference_composite = s1_collection \
    .filterDate(reference_start, reference_end) \
    .median() \
    .clip(aoi)

target_composite = s1_collection \
    .filterDate(target_start, target_end) \
    .median() \
    .clip(aoi)

# Calcular número de imágenes en cada composición
n_ref = s1_collection.filterDate(reference_start, reference_end).size().getInfo()
n_target = s1_collection.filterDate(target_start, target_end).size().getInfo()

print(f"\nImágenes en período de referencia: {n_ref}")
print(f"Imágenes en período de análisis: {n_target}")

In [ ]:
# Calcular diferencias
# Diferencia absoluta: target - reference
diff_vv = target_composite.select('VV_filtered').subtract(
    reference_composite.select('VV_filtered')
).rename('diff_VV')

diff_vh = target_composite.select('VH_filtered').subtract(
    reference_composite.select('VH_filtered')
).rename('diff_VH')

# Diferencia relativa (porcentual)
rel_diff_vv = target_composite.select('VV_filtered').subtract(
    reference_composite.select('VV_filtered')
).divide(
    reference_composite.select('VV_filtered').abs()
).multiply(100).rename('rel_diff_VV')

rel_diff_vh = target_composite.select('VH_filtered').subtract(
    reference_composite.select('VH_filtered')
).divide(
    reference_composite.select('VH_filtered').abs()
).multiply(100).rename('rel_diff_VH')

# Magnitud de cambio (vector de cambio)
change_magnitude = diff_vv.pow(2).add(diff_vh.pow(2)).sqrt().rename('change_magnitude')

print("Capas de cambio calculadas:")
print("  - Diferencia absoluta VV y VH")
print("  - Diferencia relativa VV y VH")
print("  - Magnitud de cambio (vector)")

## 4. Método 2: análisis de anomalías

Calcula z-scores por píxel utilizando la media y desviación estándar de la serie histórica. Los z-scores permiten detectar valores atípicos persistentes, siguiendo la lógica de las pruebas estadísticas propuestas para series Sentinel-1 [3].

In [ ]:
# Calcular estadísticas temporales para toda la serie
mean_vv = s1_collection.select('VV_filtered').mean().clip(aoi)
std_vv = s1_collection.select('VV_filtered').reduce(ee.Reducer.stdDev()).clip(aoi)

mean_vh = s1_collection.select('VH_filtered').mean().clip(aoi)
std_vh = s1_collection.select('VH_filtered').reduce(ee.Reducer.stdDev()).clip(aoi)

# Calcular Z-score para el período de análisis
# Z = (valor - media) / desviación estándar
z_score_vv = target_composite.select('VV_filtered').subtract(mean_vv).divide(std_vv).rename('z_score_VV')
z_score_vh = target_composite.select('VH_filtered').subtract(mean_vh).divide(std_vh).rename('z_score_VH')

# Detectar anomalías significativas (|Z| > 2 = ~95% confianza)
anomaly_vv = z_score_vv.abs().gt(2).rename('anomaly_VV')
anomaly_vh = z_score_vh.abs().gt(2).rename('anomaly_VH')

# Anomalías en ambas polarizaciones
anomaly_both = anomaly_vv.And(anomaly_vh).rename('anomaly_both')

print("Análisis de anomalías completado")
print("Umbral de significancia: |Z-score| > 2 (95% confianza)")

## 5. Método 3: índice normalizado de vector de cambio (NDCV)

El NDCV resume la magnitud del cambio combinando las bandas VV y VH en una medida entre 0 y 1. Valores cercanos a 1 indican transformaciones fuertes y ayudan a resaltar eventos abruptos, como conversiones de suelo [4].

In [ ]:
# NDCV = |target - reference| / (target + reference)
# Valores cercanos a 1 indican cambio fuerte, cercanos a 0 indican no cambio

def calculate_ndcv(band_name):
    """Calcula el Índice Normalizado de Vector de Cambio para una banda"""
    ref_band = reference_composite.select(band_name)
    target_band = target_composite.select(band_name)
    
    # Convertir de dB a lineal para el cálculo
    ref_linear = ee.Image(10).pow(ref_band.divide(10))
    target_linear = ee.Image(10).pow(target_band.divide(10))
    
    # NDCV
    ndcv = target_linear.subtract(ref_linear).abs().divide(
        target_linear.add(ref_linear)
    )
    
    return ndcv

ndcv_vv = calculate_ndcv('VV_filtered').rename('NDCV_VV')
ndcv_vh = calculate_ndcv('VH_filtered').rename('NDCV_VH')

# NDCV combinado (promedio de ambas polarizaciones)
ndcv_combined = ndcv_vv.add(ndcv_vh).divide(2).rename('NDCV_combined')

# Umbral de cambio significativo (típicamente > 0.3)
change_mask = ndcv_combined.gt(0.3).rename('change_mask')

print("NDCV calculado")
print("Umbral de cambio: NDCV > 0.3")

## 6. Clasificación de tipos de cambio

Integra los tres métodos para clasificar cada píxel en categorías sencillas: aumento, disminución o variabilidad mixta. Puedes ajustar umbrales en función del cultivo y de la experiencia en campo, tal como sugieren los ejercicios prácticos del tutorial [6].

In [ ]:
# Clasificar cambios basados en dirección y magnitud
# Clases:
# 0: Sin cambio
# 1: Aumento fuerte (posible crecimiento vegetación, inundación)
# 2: Disminución fuerte (posible cosecha, deforestación, sequía)
# 3: Cambio moderado positivo
# 4: Cambio moderado negativo

# Umbrales (en dB)
strong_threshold = 3  # cambio > 3 dB es significativo
moderate_threshold = 1.5  # cambio entre 1.5-3 dB es moderado

# Inicializar con clase 0 (sin cambio)
change_classification = ee.Image(0).clip(aoi)

# Usar VV para clasificación (más sensible a cambios estructurales)
diff_vv_value = diff_vv

# Aumento fuerte
change_classification = change_classification.where(
    diff_vv_value.gt(strong_threshold),
    1
)

# Disminución fuerte
change_classification = change_classification.where(
    diff_vv_value.lt(-strong_threshold),
    2
)

# Cambio moderado positivo
change_classification = change_classification.where(
    diff_vv_value.gt(moderate_threshold).And(diff_vv_value.lte(strong_threshold)),
    3
)

# Cambio moderado negativo
change_classification = change_classification.where(
    diff_vv_value.lt(-moderate_threshold).And(diff_vv_value.gte(-strong_threshold)),
    4
)

change_classification = change_classification.rename('change_class')

print("Clasificación de cambios:")
print("  0: Sin cambio")
print("  1: Aumento fuerte (> 3 dB)")
print("  2: Disminución fuerte (< -3 dB)")
print("  3: Aumento moderado (1.5-3 dB)")
print("  4: Disminución moderada (-1.5 a -3 dB)")

## 7. Estadísticas de cambio por municipio

Agrega los resultados por municipio para calcular superficie afectada, intensidad media y porcentaje de cada categoría. Estas métricas son fáciles de explicar a equipos de planificación y siguen la estructura usada en estudios de cambio no supervisado [5].

In [ ]:
# Función para extraer estadísticas de cambio por municipio
def extract_change_stats(municipio_geom, municipio_name, departamento):
    """Extrae estadísticas de cambio para un municipio"""
    
    # Estadísticas de diferencias
    stats = ee.Dictionary({
        'municipio': municipio_name,
        'departamento': departamento
    })
    
    # Diferencia media VV y VH
    diff_stats = diff_vv.addBands(diff_vh).reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ),
        geometry=municipio_geom,
        scale=10,
        maxPixels=1e9
    )
    
    stats = stats.combine(diff_stats)
    
    # Estadísticas de NDCV
    ndcv_stats = ndcv_combined.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=municipio_geom,
        scale=10,
        maxPixels=1e9
    )
    
    stats = stats.combine(ndcv_stats)
    
    # Porcentaje de área con cambio
    area_total = municipio_geom.area().divide(10000)  # en hectáreas
    area_cambio = change_mask.multiply(ee.Image.pixelArea()).divide(10000).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=municipio_geom,
        scale=10,
        maxPixels=1e9
    ).get('change_mask')
    
    porcentaje_cambio = ee.Number(area_cambio).divide(area_total).multiply(100)
    
    stats = stats.set('area_total_ha', area_total)
    stats = stats.set('area_cambio_ha', area_cambio)
    stats = stats.set('porcentaje_cambio', porcentaje_cambio)
    
    return stats

print("Extrayendo estadísticas por municipio...")
print("(Este proceso puede tomar varios minutos)")

In [ ]:
# Extraer estadísticas para todos los municipios
stats_list = []

for idx, row in municipios_seleccionados.iterrows():
    geom = row.geometry
    
    # Convertir a geometría EE
    if geom.geom_type == 'Polygon':
        coords = [list(geom.exterior.coords)]
        ee_geom = ee.Geometry.Polygon(coords)
    else:
        coords = [list(poly.exterior.coords) for poly in geom.geoms]
        ee_geom = ee.Geometry.MultiPolygon(coords)
    
    # Extraer estadísticas
    stats = extract_change_stats(
        ee_geom,
        row['mpio_cnmbr'],
        row['dpto_cnmbr']
    ).getInfo()
    
    stats_list.append(stats)
    print(f"  Procesado: {row['mpio_cnmbr']}, {row['dpto_cnmbr']}")

# Crear DataFrame
stats_df = pd.DataFrame(stats_list)

# Ordenar por porcentaje de cambio
stats_df = stats_df.sort_values('porcentaje_cambio', ascending=False)

print("\n" + "="*80)
print("ESTADÍSTICAS DE CAMBIO POR MUNICIPIO")
print("="*80)
print(stats_df.to_string(index=False))

## 8. Visualización de resultados

Muestra mapas interactivos con las diferencias de VV, el NDCV y la clasificación final. Las leyendas claras ayudan a los lectores a relacionar colores con procesos agrícolas específicos.

In [ ]:
# Crear mapa interactivo
Map = geemap.Map(
    center=[parametros['centroide_lat'], parametros['centroide_lon']],
    zoom=9
)

# Agregar municipios
Map.add_gdf(municipios_seleccionados, layer_name="Municipios", style={'fillOpacity': 0})

# Parámetros de visualización
vis_sar = {'min': -25, 'max': 0, 'palette': ['blue', 'white', 'green']}
vis_diff = {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}
vis_ndcv = {'min': 0, 'max': 0.6, 'palette': ['white', 'yellow', 'orange', 'red']}
vis_class = {
    'min': 0,
    'max': 4,
    'palette': ['gray', 'blue', 'red', 'lightblue', 'orange']
}

# Agregar capas
Map.addLayer(reference_composite.select('VV_filtered'), vis_sar, 'Referencia VV', shown=False)
Map.addLayer(target_composite.select('VV_filtered'), vis_sar, 'Análisis VV', shown=False)
Map.addLayer(diff_vv, vis_diff, 'Diferencia VV', shown=True)
Map.addLayer(ndcv_combined, vis_ndcv, 'NDCV', shown=True)
Map.addLayer(change_classification, vis_class, 'Clasificación de Cambios', shown=True)
Map.addLayer(change_mask.selfMask(), {'palette': 'red'}, 'Máscara de Cambio (NDCV>0.3)', shown=False)

# Agregar leyenda
legend_dict = {
    'Sin cambio': 'gray',
    'Aumento fuerte (>3dB)': 'blue',
    'Disminución fuerte (<-3dB)': 'red',
    'Aumento moderado': 'lightblue',
    'Disminución moderada': 'orange'
}
Map.add_legend(legend_dict=legend_dict, title='Tipos de Cambio')

Map

## 9. Análisis de patrones temporales

Extrae series temporales para municipios o puntos de interés y grafica ventanas móviles. Esto permite distinguir entre cambios graduales por manejo agrícola y picos repentinos ligados a eventos climáticos [6].

In [ ]:
# Crear series temporales mensuales para un punto de interés
# (Usar el centroide del área de estudio como ejemplo)

point_of_interest = ee.Geometry.Point(
    [parametros['centroide_lon'], parametros['centroide_lat']]
)

# Función para extraer valores en un punto
def extract_point_values(image):
    date = image.date().format('YYYY-MM-dd')
    values = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point_of_interest.buffer(100),  # buffer de 100m
        scale=10
    )
    return ee.Feature(None, {
        'date': date,
        'VV': values.get('VV_filtered'),
        'VH': values.get('VH_filtered')
    })

# Extraer serie temporal
time_series_fc = s1_collection.map(extract_point_values)
time_series_data = time_series_fc.aggregate_array('date').getInfo()
vv_values = time_series_fc.aggregate_array('VV').getInfo()
vh_values = time_series_fc.aggregate_array('VH').getInfo()

# Crear DataFrame
ts_df = pd.DataFrame({
    'date': pd.to_datetime(time_series_data),
    'VV': vv_values,
    'VH': vh_values
})

ts_df = ts_df.sort_values('date')

print(f"Serie temporal extraída: {len(ts_df)} observaciones")

In [ ]:
# Visualizar serie temporal
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# VV
axes[0].plot(ts_df['date'], ts_df['VV'], 'b-', linewidth=1, label='VV')
axes[0].scatter(ts_df['date'], ts_df['VV'], s=20, c='blue', alpha=0.5)
axes[0].axvline(pd.to_datetime(reference_start), color='green', linestyle='--', label='Inicio Referencia')
axes[0].axvline(pd.to_datetime(reference_end), color='green', linestyle='--', label='Fin Referencia')
axes[0].axvline(pd.to_datetime(target_start), color='red', linestyle='--', label='Inicio Análisis')
axes[0].axvline(pd.to_datetime(target_end), color='red', linestyle='--', label='Fin Análisis')
axes[0].set_ylabel('Backscatter VV (dB)')
axes[0].set_title('Serie Temporal Sentinel-1 - Punto Central del Área de Estudio')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# VH
axes[1].plot(ts_df['date'], ts_df['VH'], 'r-', linewidth=1, label='VH')
axes[1].scatter(ts_df['date'], ts_df['VH'], s=20, c='red', alpha=0.5)
axes[1].axvline(pd.to_datetime(reference_start), color='green', linestyle='--')
axes[1].axvline(pd.to_datetime(reference_end), color='green', linestyle='--')
axes[1].axvline(pd.to_datetime(target_start), color='red', linestyle='--')
axes[1].axvline(pd.to_datetime(target_end), color='red', linestyle='--')
axes[1].set_ylabel('Backscatter VH (dB)')
axes[1].set_xlabel('Fecha')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.savefig('data/time_series.png', dpi=150, bbox_inches='tight')
plt.show()

print("Serie temporal guardada en: data/time_series.png")

## 10. Exportar resultados

Guarda las tablas y archivos espaciales en `data/` con nombres descriptivos. Incluye metadatos sobre fechas, umbrales y métodos para facilitar auditorías y comparaciones futuras.

In [ ]:
# Guardar estadísticas de cambio
stats_df.to_csv('data/estadisticas_cambio_municipios.csv', index=False)
print("Estadísticas guardadas en: data/estadisticas_cambio_municipios.csv")

# Guardar serie temporal
ts_df.to_csv('data/serie_temporal_punto_central.csv', index=False)
print("Serie temporal guardada en: data/serie_temporal_punto_central.csv")

# Guardar parámetros del análisis de cambios
change_analysis_params = {
    'reference_period': {'start': reference_start, 'end': reference_end},
    'target_period': {'start': target_start, 'end': target_end},
    'n_images_reference': n_ref,
    'n_images_target': n_target,
    'strong_change_threshold_db': strong_threshold,
    'moderate_change_threshold_db': moderate_threshold,
    'ndcv_threshold': 0.3,
    'z_score_threshold': 2
}

with open('data/change_analysis_params.json', 'w') as f:
    json.dump(change_analysis_params, f, indent=2)

print("Parámetros de análisis guardados en: data/change_analysis_params.json")

## Resumen

Este notebook combina tres métodos de detección de cambios, clasifica los resultados y genera métricas y visualizaciones por municipio. Los archivos exportados sirven como insumo directo para la etapa de comunicación de resultados.

**Próximo notebook:** `04_visualizacion_interpretacion.ipynb` crea gráficos, mapas y mensajes clave para difundir los hallazgos.

### Referencias bibliográficas

[1] K. Conradsen, A. A. Nielsen, J. Schou y H. Skriver, “A Test Statistic in the Complex Wishart Distribution and Its Application to Change Detection in Polarimetric SAR Data,” IEEE Trans. Geosci. Remote Sens., vol. 41, no. 1, pp. 4–19, 2003, doi: 10.1109/TGRS.2002.808066.

[2] M. J. Canty, A. A. Nielsen, H. Skriver y K. Conradsen, “Statistical Analysis of Changes in Sentinel-1 Time Series on the Google Earth Engine,” Remote Sensing, vol. 12, no. 1, p. 46, 2020, doi: 10.3390/rs12010046.

[3] A. A. Nielsen, K. Conradsen y H. Skriver, “Omnibus Test for Change Detection in a Time Sequence of Polarimetric SAR Data,” en IEEE Int. Geosci. Remote Sens. Symp., 2015, pp. 3398–3401, doi: 10.1109/IGARSS.2015.7326538.

[4] M. J. Canty, *Image Analysis, Classification and Change Detection in Remote Sensing, with Algorithms for Python*, 4.ª ed., Boca Raton, FL, USA: CRC Press, 2019.

[5] Y. Bazi, L. Bruzzone y F. Melgani, “An Unsupervised Approach Based on the Generalized Gaussian Model to Automatic Change Detection in Multitemporal SAR Images,” IEEE Trans. Geosci. Remote Sens., vol. 43, no. 4, pp. 874–887, 2005, doi: 10.1109/TGRS.2004.842441.

[6] Google Earth Engine Developers, “Detecting Changes in Sentinel-1 Imagery, Part 3,” Google Developers, Mountain View, CA, USA, 2024. Disponible en: https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3
